# Calculo de Probabilidade de Viagem - Método 2
Este cálculo está baseado no Paper:

*Preparedness and vulnerability of African countries against importations of COVID-19: a modelling study - Gilbert et al. (2020)* 
 


In [17]:
import pandas as pd
import numpy as np
import random


In [23]:
df_municipio = pd.read_csv("data/integrado/municipio.csv")
df_municipio = df_municipio[df_municipio.pais == 'BRA']
df_fluxo_aereo = pd.read_csv("data/integrado/fluxo_aereo.csv")
df_fluxo_rodoviario = pd.read_csv("data/integrado/fluxo_rodoviario.csv")

In [26]:
#Simular o número de infectados de uma cidade
#De 0 a 5000
# df_municipio["infectados"] = np.random.randint(0, 500, df_municipio.shape[0])
df_municipio["infectados"] = df_municipio["populacao_2021"]*random.uniform(0, 0.05)
df_municipio.infectados = df_municipio.infectados.astype(int)

In [27]:
df_municipio

,cod_mun,nome_mun,latitude,longitude,cod_uf,populacao_2011,populacao_2021,area,densidade_2021,cod_reg_saude,nome_uf,uf,pais,infectados
0,5200050,Abadia de Goiás,-16.75730,-49.4412,52,7023.0,9158.0,143.357,63.882475,52001.0,Goiás,GO,BRA,385
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,31,6724.0,7022.0,880.461,7.975367,31074.0,Minas Gerais,MG,BRA,295
2,5200100,Abadiânia,-16.19700,-48.7057,52,16088.0,20873.0,1044.555,19.982672,52011.0,Goiás,GO,BRA,878
3,3100203,Abaeté,-19.15510,-45.4444,31,22716.0,23263.0,1817.067,12.802500,31024.0,Minas Gerais,MG,BRA,979
4,1500107,Abaetetuba,-1.72183,-48.8788,15,142785.0,160439.0,1610.654,99.611090,15011.0,Pará,PA,BRA,6755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,2933604,Xique-Xique,-10.82300,-42.7245,29,45599.0,46562.0,5079.662,9.166358,29010.0,Bahia,BA,BRA,1960
5566,2517407,Zabelê,-8.07901,-37.1057,25,2093.0,2269.0,106.811,21.243130,25005.0,Paraíba,PB,BRA,95
5567,3557154,Zacarias,-21.05060,-50.0552,35,2365.0,2784.0,319.056,8.725741,35156.0,São Paulo,SP,BRA,117
5568,2114007,Zé Doca,-3.27014,-45.6553,21,51615.0,52190.0,2140.109,24.386608,21019.0,Maranhão,MA,BRA,2197


In [28]:
#Calcular o fluxo total de saida de cada cidade
#Aéreo 

df_municipio["fluxo"] = df_municipio.cod_mun.apply(lambda cod_mun: df_fluxo_aereo[df_fluxo_aereo.cod_mun_destino == cod_mun].total_passagens_2020.sum())
# df_fluxo_aereo[df_fluxo_aereo.cod_mun_destino == 3550308].total_passagens_2020.sum()

In [29]:
def info_cidade(codigo_municipio, df_municipio, df_fluxo_aereo, df_fluxo_rodoviario):
    
    #Consultando valores nos dataframe a partir do código da cidade
    total_populacao = df_municipio[df_municipio.cod_mun == codigo_municipio].populacao_2021
    
    #TODO: Implementar o sentido do fluxo
    # df_aereo = df_fluxo_aereo[df_fluxo_aereo.cod_mun_origem == codigo_municipio][['cod_mun_destino', 'total_passagens_2020']].rename(columns={"cod_mun_destino": "cod_mun"})
    df_aereo = df_fluxo_aereo[df_fluxo_aereo.cod_mun_destino == codigo_municipio][['cod_mun_origem', 'total_passagens_2020']].rename(columns={"cod_mun_origem": "cod_mun"})
    
    
    # df_rod_1 = df_fluxo_rodoviario[(df_fluxo_rodoviario.cod_mun_a == codigo_municipio)][['cod_mun_b', 'freq_saida_2016']].rename(columns={"cod_mun_b": "cod_mun"})
    # df_rod_2 = df_fluxo_rodoviario[(df_fluxo_rodoviario.cod_mun_b == codigo_municipio)][['cod_mun_a', 'freq_saida_2016']].rename(columns={"cod_mun_a": "cod_mun"})
    # df_rod = df_rod_1.append(df_rod_2, ignore_index=True).sort_values(by=['freq_saida_2016'])
    
    # df_fluxo = df_aereo.merge(df_rod[['cod_mun', 'freq_saida_2016']], how='outer', left_on='cod_mun', right_on='cod_mun')
    df_fluxo = df_aereo #TODO: Remover esta linha quando adicionar o rodoviario -> Bug do código 0
    df_fluxo.fillna(0, inplace = True)

    # df_fluxo["total_pop"] = df_fluxo["cod_mun"].apply(lambda x: df_municipio.loc[df_municipio.cod_mun==x, 'populacao_2021'].values[0])
    df_fluxo = df_fluxo.merge(df_municipio[['cod_mun', 'populacao_2021','infectados', 'fluxo']], how='left', left_on='cod_mun', right_on='cod_mun')
    
    lista_fluxo = list(df_fluxo.itertuples(index=False, name=None))
    
    return (total_populacao, lista_fluxo)

In [30]:
lista_mun = df_municipio["cod_mun"].tolist()

In [31]:
probabilidade_exportar = {}
probabilidade_importar = {}

#Calcular todas as probabilidades de importação
for cod_mun_d in lista_mun:

    prob_importar_cidade = {}
    total_populacao, lista_fluxo= info_cidade(cod_mun_d, df_municipio, df_fluxo_aereo, df_fluxo_rodoviario)

    partial_risk_den = sum((inf/pop)*aereo for cod, aereo, pop, inf, fluxo in lista_fluxo)

    #Calular probabilidade única de importação
    for cod_mun_o, fluxo_aereo, pop, infectados, fluxo_total in lista_fluxo:
        fluxo_para_cidade = fluxo_aereo

        if(fluxo_total!=0):
            partial_risk_num = ((infectados/pop)*fluxo_para_cidade)*(fluxo_para_cidade/fluxo_total) 
        else:
            partial_risk_num = 0
        
        partial_risk = (partial_risk_num / partial_risk_den) if partial_risk_num!=0 else 0
        prob_importar_cidade[cod_mun_o] = partial_risk

        if(cod_mun_o==2605459 and cod_mun_d==2611606):
            print("nume ", partial_risk_num)
            print("den ", partial_risk_den)
        # probabilidade_exportar[codigo]
        # print(codigo, partial_risk)

        #Adicionando a probabilidade de exportar casos a cidade
        if not (cod_mun_o in probabilidade_exportar):
            probabilidade_exportar[cod_mun_o] = {}
        probabilidade_exportar[cod_mun_o][cod_mun_d] = partial_risk


    probabilidade_importar[cod_mun_d] = prob_importar_cidade
    

nume  400.5440855651907
den  38635.57530964567


### Verificando para Recife 

Probabilidade de Importação

In [39]:
re = probabilidade_importar[2611606]
re_prob = list(re.items())

In [40]:
sum(n for _, n in re_prob)

0.0696169554508777

In [41]:
re_prob.sort(key=lambda x:-x[1])
for cod_mun, prob in re_prob:
    nome = df_municipio.loc[df_municipio.cod_mun==cod_mun, 'nome_mun'].values[0]
    print(f"{nome} - {prob}")

São Paulo - 0.02347044359207951
Fernando de Noronha - 0.010367234921572186
Fortaleza - 0.0065465105733214235
Brasília - 0.006214908327920605
Rio de Janeiro - 0.004723129787616756
Petrolina - 0.0027043752800933597
Manaus - 0.0024009555902306345
Salvador - 0.0022780675844878157
Campinas - 0.0011525918446992779
Cuiabá - 0.0010007087740325914
Porto Alegre - 0.0009623725193201791
Goiânia - 0.0008767978872601755
Belém - 0.0008321620049176458
Teresina - 0.0008126359851798272
São Luís - 0.0006902567044017235
Belo Horizonte - 0.000628941307558346
Curitiba - 0.000546765849969894
Campo Grande - 0.0004891335131036758
Itajaí - 0.0004086244904846082
Vitória - 0.0003740358715465758
Porto Velho - 0.0002995658990170237
Florianópolis - 0.0002798957413021304
Uberlândia - 0.00027415286872077836
Aracaju - 0.0002534429449060345
Juazeiro do Norte - 0.0002451346769580046
Foz do Iguaçu - 0.00016971759587070212
Palmas - 0.00015394930759656767
Maringá - 0.00010640938920670384
Boa Vista - 8.58033269862278e-05
Mar

In [48]:
df_municipio[df_municipio.cod_mun == 3550308] #Fernando de noronha
# df_municipio[df_municipio.cod_mun == 2611101] #petrolina


,cod_mun,nome_mun,latitude,longitude,cod_uf,populacao_2011,populacao_2021,area,densidade_2021,cod_reg_saude,nome_uf,uf,pais,infectados,fluxo
4853,3550308,São Paulo,-23.5329,-46.6395,35,11316119.0,12396372.0,1521.202,8149.063701,35016.0,São Paulo,SP,BRA,521995,3834935.0


Probabilidade de Exportação

In [46]:
# re_ex = probabilidade_exportar[2611606] # Recife
re_ex = probabilidade_exportar[3550308] # Sao Paulo
re_prob_ex = list(re_ex.items())
re_prob_ex.sort(key=lambda x:-x[1])
for cod_mun, prob in re_prob_ex:
    nome = df_municipio.loc[df_municipio.cod_mun==cod_mun, 'nome_mun'].values[0]
    print(f"{nome} - {prob}")

Salvador - 0.025407008799194976
Rio de Janeiro - 0.025085499269571886
Porto Alegre - 0.025001455739804758
Recife - 0.02347044359207951
Florianópolis - 0.016914585708016915
Belo Horizonte - 0.014862360335670873
Fortaleza - 0.013720259817278058
Curitiba - 0.011074874911522217
Maceió - 0.010986039347705298
Brasília - 0.010486192812392724
Vitória - 0.010294687396756303
Goiânia - 0.009576593532734923
Juazeiro do Norte - 0.008603129626199702
Cuiabá - 0.0084939764743066
Natal - 0.007836102090586623
Aracaju - 0.007534818254679868
Itajaí - 0.007200105824028031
João Pessoa - 0.0066736948548976344
Campo Grande - 0.00529955849303017
Teresina - 0.005278997668767875
São Luís - 0.005271173691618045
Manaus - 0.004928095295559178
Foz do Iguaçu - 0.004084586728217058
Petrolina - 0.0038553366415304954
Belém - 0.0035768453304730035
Uberlândia - 0.003162640406982979
Maringá - 0.00292932307473416
Palmas - 0.0013646691230167296
Montes Claros - 0.0011744791220629576
Rio Branco - 0.0006993164338790619
Porto Ve

In [47]:
sum(n for _, n in re_prob_ex)

0.2870013220813957